### mysql과 python 연동        
1. mysql과 연동할 수 있는 기능을 가진 라이브러리 설치
    - python에서 라이브러리들이 저장되어 있는 공간
    - R에서는 CRAN에 저장소가 있다면 python에서는 pip 저장소에 저장되어 있다.
    - 라이브러리 설치 명령어 : pip install 라이브러리명
    - mysql과 연동하는 라이브러리의 이름 : pymysql
    - pip install pymysql 
2. 라이브러리 로드 
    - import 라이브러리명 
    - from 라이브러리명 import (Class|function|module)
    - as 별칭 -> 별칭을 통해 라이브러리를 간단하게 쓰기 위해
3. mysql 서버의 정보를 입력하고 연결 
    - DB서버의 주소 : host 
        - localhost, 127.0.0.1 : 내 컴퓨터 
        - 외부의 주소 (ex: 172.168.1.40)
    - Port (방화벽 문)
        - mysql의 기본포트 : 3306
    - user 
        - mysql에 연결할 때 사용하는 사용자의 이름
        - 기본 계정 (root)
    - password
        - mysql에 연결할 때 사용하는 사용자의 비밀번호
    - db
        - mysql 서버에서 사용할 Database의 이름

4. DB서버와 python공간 사이에 가상 공간을 생성 
    - cursor는 가상 공간의 이름 
5. sql 쿼리문을 이용하여 cursor에 질의를 보낸다.
    - 데이터의 변환 질의 (insert, update, delete)
        - 질의 보내기 (execute())
        - DB서버의 데이터 변경 (commit())                         # github에서 push
        - DB서버와의 연결 종료 (close())
    - 데이터의 로드 질의 (selcet)
        - 질의 보내기 (execute())
        - 질의에 대한 결과물 받아오기 (fetchall())             # fetchall 후 가상공간에서 데이터는 사라진다.
        - DB서버와의 연결 종료 (close())                          # 연결종료 순간 가상공간도 없어진다.

In [3]:
## 1. 라이브러리 설치 
!pip install pymysql

In [10]:
## 2. 라이브러리 로드 
import pymysql

In [5]:
# mysql 연결시 cryptography 에러 발생
!pip install cryptography

In [11]:
## 3. 서버 정보 입력
_db = pymysql.connect(
    host = 'localhost',
    port = 3306,
    user = 'root',
    password = '1234',
    db = 'ubion'
)

In [49]:
## 4. cursor 생성 (기본값 설정)
cursor = _db.cursor()
# cursor2 생성 (옵션변경)
cursor2 = _db.cursor(pymysql.cursors.DictCursor)

# cursor의 기본 설정대로 생성 시, 데이터 로드하는 과정에서 받아오는 데이터 타입이 2차원 tuple 형태
# DictCursor 이용 시, 받아오는 데이터 타입은 list 안에 dict 형태 

In [14]:
## 5. sql 쿼리문 
# query문을 작성 -> 문자형 데이터
query = """
    select 
    *
    from
    tran_1
"""
# execute()를 이용하여 query문을 cursor에게 보낸다.
cursor.execute(query)
cursor2.execute(query)

5000

In [ ]:
cursor.fetchall()    # 저장해주지 않았으므로 두번 실행 시 ()로 비어있다는 결과가 나온다. fechall() 후 Cursor는 클리어된다.

In [ ]:
cursor2.fetchall()

In [11]:
# DB 서버와 연결 종료
_db.close()

In [15]:
# DB서버에 데이터들 수정
query = """
    update
    tran_1
    set
    price = 10000
    where 
    transaction_id = 'T0000000113'
"""

In [19]:
# 데이터 변환 시 흐름 : execute() -> commit()
cursor.execute(query)                               # >1        하나 바뀌었다. 조건이 2개였으면 2라고 떴을 것.

1

In [22]:
select_query = """
    select * from tran_1 limit 5
"""
cursor.execute(select_query)
cursor.fetchall()

(('T0000000113', 10000, '2019-02-01 01:36:57', 'PL563502'),
 ('T0000000114', 50000, '2019-02-01 01:37:23', 'HD678019'),
 ('T0000000115', 120000, '2019-02-01 02:34:19', 'HD298120'),
 ('T0000000116', 210000, '2019-02-01 02:47:23', 'IK452215'),
 ('T0000000117', 170000, '2019-02-01 04:33:46', 'PL542865'))

In [14]:
# cursor에서 변경한 내역을 DB 서버에 보낸다. (cursor와 DB를 동기화)
_db.commit()

In [13]:
### 해당하는 과정을 함수 형태로   --->  이 함수들이 모여 모듈로 만드면 모듈만 불러와서 함수 사용할 수 있다. 
## select 문을 사용할 때의 함수 생성 
def query_select(_sql):
    # cursor에 _sql의 질의를 보낸다.
    cursor.execute(_sql)
    # 결과물을 돌려받는다.
    result = cursor.fetchall()
    return result

In [19]:
select1 = """
    select * from tran_1 limit 5
"""

In [20]:
query_select(select1)

(('T0000000113', 10000, '2019-02-01 01:36:57', 'PL563502'),
 ('T0000000114', 50000, '2019-02-01 01:37:23', 'HD678019'),
 ('T0000000115', 120000, '2019-02-01 02:34:19', 'HD298120'),
 ('T0000000116', 210000, '2019-02-01 02:47:23', 'IK452215'),
 ('T0000000117', 170000, '2019-02-01 04:33:46', 'PL542865'))

In [21]:
## insert, update, delete문을 이용할 때 사용할 함수 생성 (원래 excute와 commit 같은 함수에 들어가면 안된다. 연습이니 그냥 한다.)
def cud_query(_sql):
    # _sql를 cursor에 excute()
    cursor.execute(_sql)
    # DB서버와 동기화
    _db.commit()
    return 'Query Ok'

In [22]:
delete1 = """
    delete 
    from 
    tran_1
    where
    transaction_id = 'T0000000113'
"""

In [24]:
cud_query(delete1)

'Query Ok'

In [35]:
query = """
    Select * from tran_1
"""
# query 문자열이 select로 시작하는가?
# case1 (문자를 잘라서 확인)
print(query.strip()[:6].lower() == 'select')
# case2 (공백을 기준으로 문자열을 나눠준다.)
print(query.split()[0].lower() == 'select')
# case3 (문자의 시작이 특정 단어로 시작하는가?)
print(query.lower().strip().startswith('select'))

False
False
False


In [38]:
## query_select() 함수와 cud_query() 함수를 결합하여 하나의 함수 생성
def sql_query(_sql):
    # cursor에 질의를 보낸다.
    cursor.execute(_sql)
    # _sql에 입력되는 인자의 값이 select문이라면?
    if _sql.lower().strip().startswith('select'):
        # select문일 때 실행할 코드
        result = cursor.fetchall()
    else:
        # select문이 아닐 때 실행할 코드 
        _db.commit()
        result = 'Query Ok'
    
    return result

In [40]:
select2 = """
    SELECT * from tran_1 where price > 20000 limit 10
"""
sql_query(select2)

(('T0000000114', 50000, '2019-02-01 01:37:23', 'HD678019'),
 ('T0000000115', 120000, '2019-02-01 02:34:19', 'HD298120'),
 ('T0000000116', 210000, '2019-02-01 02:47:23', 'IK452215'),
 ('T0000000117', 170000, '2019-02-01 04:33:46', 'PL542865'),
 ('T0000000118', 120000, '2019-02-01 04:36:50', 'GD372798'),
 ('T0000000119', 180000, '2019-02-01 05:25:18', 'IK782104'),
 ('T0000000120', 50000, '2019-02-01 06:22:39', 'IK624245'),
 ('T0000000121', 210000, '2019-02-01 06:40:14', 'HD218970'),
 ('T0000000122', 85000, '2019-02-01 08:10:23', 'AS912483'),
 ('T0000000123', 50000, '2019-02-01 09:32:27', 'OA336650'))

In [41]:
delete2 = """
    delete from tran_1 where transaction_id = 'T0000000115'
"""
sql_query(delete2)

'Query Ok'

In [52]:
select3 = """
    select * from tran_1 where price >= %s and price <= %s
"""

#select3 = """
#    select * from tran_1 where between %s and %s                       # 왜 에러 뜨는 지 확인할 것.
# """
value = [500000, 700000]                            # 튜플로도 가능 
cursor.execute(select3,value)
cursor.fetchall()



(('T0000000513', 570000, '2019-02-11 10:28:04', 'GD649455'),
 ('T0000001026', 630000, '2019-02-24 11:58:34', 'AS108195'),
 ('T0000003754', 675000, '2019-05-08 09:49:08', 'HD096953'),
 ('T0000003841', 570000, '2019-05-10 21:33:32', 'TS244750'))

In [53]:
def sql_query(_sql, *_values):
    # cursor에 질의를 보낸다.
    cursor.execute(_sql, _values)
    # _sql에 입력되는 인자의 값이 select문이라면?
    if _sql.lower().strip().startswith('select'):
        # select문일 때 실행할 코드
        result = cursor.fetchall()
    else:
        # select문이 아닐 때 실행할 코드 
        _db.commit()
        result = 'Query Ok'
    
    return result

In [54]:
sql_query(select3,500000,700000)

(('T0000000513', 570000, '2019-02-11 10:28:04', 'GD649455'),
 ('T0000001026', 630000, '2019-02-24 11:58:34', 'AS108195'),
 ('T0000003754', 675000, '2019-05-08 09:49:08', 'HD096953'),
 ('T0000003841', 570000, '2019-05-10 21:33:32', 'TS244750'))

In [55]:
select4 = """
    select * from tran_1 limit 3
"""
sql_query(select4)                             # values 값 없는 쿼리문도 돌아간다. (values값 넣어주지 않아도 돌아간다.)

(('T0000000114', 50000, '2019-02-01 01:37:23', 'HD678019'),
 ('T0000000116', 210000, '2019-02-01 02:47:23', 'IK452215'),
 ('T0000000117', 170000, '2019-02-01 04:33:46', 'PL542865'))

In [93]:
## mysql server와의 연동을 하는 class 선언                     # 녹음 16:46
class MyDB :
    # 생성자 함수  (DB서버의 정보를 입력)
    def __init__(
            self,
            _host = 'localhost',
            _port = 3306,
            _user = 'root',
            _pw = '1234',
            _db = 'ubion'                              # 기본값 입력. 아무것도 넣지 않으면 내 컴퓨터 DB 접속
    ):
        self.host = _host
        self.port = _port
        self.user = _user
        self.pw = _pw
        self.db = _db
    
    def sql_query(self, _sql, *_values):
        # DB 서버와의 연결
        mydb = pymysql.connect(
            host = self.host,
            port = self.port,
            user = self.user,
            password = self.pw,
            db = self.db
        )
        # cursor 생성
        cursor = mydb.cursor(pymysql.cursors.DictCursor)
        # _sql, _values를 이용하여 cursor에 질의를 던진다.
        cursor.execute(_sql, _values)
        # _sql이 select문인가 확인 
        if _sql.lower().strip().startswith('select'):
            result = cursor.fetchall()
        else:
            mydb.commit()
            result = 'Query OK'
        # DB 서버와의 연결 종료
        mydb.close()
        # 결과값을 되돌려준다.
        return result

In [94]:
# MyDB Class 생성
db1 = MyDB()

In [96]:
# db1 : 서버의 정보가 내 컴퓨터 
select1 = """
    select * from tran_1 limit %s
"""
db1.sql_query(select1, 3)

[{'transaction_id': 'T0000000114',
  'price': 50000,
  'payment_date': '2019-02-01 01:37:23',
  'customer_id': 'HD678019'},
 {'transaction_id': 'T0000000116',
  'price': 210000,
  'payment_date': '2019-02-01 02:47:23',
  'customer_id': 'IK452215'},
 {'transaction_id': 'T0000000117',
  'price': 170000,
  'payment_date': '2019-02-01 04:33:46',
  'customer_id': 'PL542865'}]

In [98]:
## 외부의 컴퓨터 접속 
db2 = MyDB(
    _host = '172.30.1.55',
    _port = 3306,
    _user = 'ubion',
    _pw = '1234',
    _db = 'ubion'
)

In [106]:
select1 = """
    select * from corona limit 5
"""
db2.sql_query(select1)

delete1 = """
    delete from corona 
"""
db2.sql_query(delete1)

()

In [102]:
db3 = MyDB(
    _host = 'moons-86.iptime.org',
    _port = 3306,
    _user = 'ubion',
    _pw = '1234',
    _db = 'ubion'
)

In [107]:
select2 = """
    select * from user where name = 'kim'
"""
db3.sql_query(select2)

[{'id': 'kys', 'password': '1234', 'name': 'kim', 'loc': 'seuol'},
 {'id': 'team', 'password': '1234', 'name': 'kim', 'loc': 'seoul'},
 {'id': 'team2', 'password': '1234', 'name': 'kim', 'loc': 'seoul'},
 {'id': 'test11', 'password': '0111', 'name': 'kim', 'loc': 'seoul'},
 {'id': 'test22', 'password': '0111', 'name': 'kim', 'loc': 'seoul'}]